In [1]:
%load_ext autoreload

In [30]:
%autoreload 2
from tqdm import tqdm_notebook as tqdm
from readers import DstcDatasetReader
from allennlp.data import Vocabulary
from allennlp.data.iterators import BucketIterator, BasicIterator

# read data
reader = DstcDatasetReader()
train_dataset = reader.read("data/train") 

# build the vocab
vocab = Vocabulary.from_instances(train_dataset)

# litmus test
iterator = BasicIterator(batch_size=5)
iterator.index_with(vocab)
batch = next(iter(iterator(train_dataset)))

16142it [05:55, 45.37it/s] 

100%|██████████| 16142/16142 [00:04<00:00, 3913.05it/s]

In [29]:
from allennlp.models import Model
from allennlp.modules.token_embedders import PretrainedBertEmbedder
from allennlp.training.trainer import Trainer
from allennlp.nn.util import get_text_field_mask

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

In [31]:
class UserIntentPredictor(Model):
    
    def __init__(self, vocab):
        super().__init__(vocab)
        
        # pretrained embedding/language model
        self.emb = PretrainedBertEmbedder("bert-base-uncased", requires_grad=False, top_layer_only=True)
        
        # layers
        self.l0 = nn.Linear(self.emb.output_dim, self.emb.output_dim)
        self.l1 = nn.Linear(self.emb.output_dim, self.emb.output_dim)
        self.l2 = nn.Linear(1, 1)
        
    def forward_turn(self, batch, turn):
        # encode current utterance
        # utter = [batch, seq, emb]
        utter, _ = self.lm(
            input_ids=batch["x_utterance_tokens"][:,turn,:],
            attention_mask=batch["x_utterance_mask"][:,turn,:]
        )
        utter = self.l0(utter)

        # individually encode description
        # desc = [desc, batch, seq, emb]
        desc = []
        num_descriptions = batch["y_descriptions_tokens"].shape[2]
        for i in range(num_descriptions):
            each_desc, _ = self.lm(
                input_ids=batch["y_descriptions_tokens"][:,turn,i,:],
                attention_mask=batch["y_descriptions_mask"][:,turn,i,:]
            )
            each_desc = self.l1(each_desc)
            desc.append(each_desc)

        # score = [batch, desc, emb]
        scores = []
        for each_desc in desc:
            utter = utter.sum(1, keepdim=True) # [b,1,e] collapse across sequence length
            each_desc = each_desc.sum(1, keepdim=True).permute(0, 2, 1) # [b,e,1]
            each_score = torch.bmm(utter, each_desc) # [b,1,1]
            scores.append(each_score)
        scores = torch.cat(scores, dim=1)

        # [batch]
        turn_mask = batch["x_turn_mask"][:,turn:turn+1]

        # score = [batch, desc]
        scores = self.l2(scores).squeeze(2)
        scores = scores * turn_mask.float()
        return scores
    
    def forward(self, **params):
        # user utterance: [batch, turn seq]
        # intent desc: [batch, turn, desc, seq]
        # active intent (list): [batch, turn, desc]
        loss_value = 0
        num_turns = params["user_utterance"]["tokens"].shape[1]

        for turn in range(num_turns):
            target_scores = batch["y_scores"][:,turn,:].float()
            predicted_scores = model(batch, turn).float()

            # calculate loss
            loss = F.mse_loss(
                predicted_scores,
                target_scores,
            )

            loss.backward()
            model_optim.step()

            loss_value += loss.item()
        return loss_value # loss per dialogue
    


model = UserIntentPredictor(vocab).to("cuda")

optimizer = optim.SGD(model.parameters(), lr=0.0001)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    num_epochs=2,
    cuda_device=0
)

trainer.train()


  0%|          | 0/251 [00:00<?, ?it/s]

> <ipython-input-31-a195e4e913a8>(54)forward()
     53         import ipdb;ipdb.set_trace();
---> 54         loss_value = 0
     55         num_turns = batch["y_scores"].shape[1]



ipdb>  params


{'user_utterance': {'tokens': tensor([[[  101,  1045,  2342,  ...,     0,     0,     0],
         [  101,  1045,  1005,  ...,     0,     0,     0],
         [  101,  2054,  2060,  ...,     0,     0,     0],
         ...,
         [  101,  1045,  2074,  ...,     0,     0,     0],
         [  101,  2748,  1010,  ...,     0,     0,     0],
         [  101,  2008,  1005,  ...,     0,     0,     0]],

        [[  101,  3531,  2393,  ...,     0,     0,     0],
         [  101,  1045,  2215,  ...,     0,     0,     0],
         [  101,  2064,  2017,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[  101,  1045,  2342,  ...,     0,     0,     0],
         [  101,  1045,  2342,  ...,     0,     0,     0],
         [  101,  2053,  2008,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,   

ipdb>  params["user_utterance"]["tokens"].shape


...

torch.Size([5, 13, 27])


ipdb>  params["intent_description"]["tokens"].shape


torch.Size([5, 13, 2, 15])


ipdb>  q


BdbQuit: 